In [13]:
import pandas as pd

# Load the dataset
data = pd.read_csv('./data/politik_merge.csv')
data.head(5)

,Judul,Waktu,Link,Content,tag1,tag2,tag3,tag4,tag5,source
0,Jokowi Kenakan Pakaian Adat Betawi di Sidang T...,16/08/2024,https://nasional.kompas.com/read/2024/08/16/11...,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",Presiden Jokowi,Jokowi,sidang tahunan MPR RI 2024,Jokowi adat Betawi sidang mpr 2024,Megawati tak hadiri sidang tahunan MPR 2024,kompas
1,Amnesty International Beberkan 6 Indikator Kri...,2024-07-18,https://nasional.tempo.co/read/1893144/amnesty...,"TEMPO.CO, Jakarta - Amnesty International Indo...",Amnesty International,Amnesty International Indonesia,Kebebasan Berpendapat,Indeks Demokrasi,Revisi UU TNI,tempo
2,"Jelang Long Weekend, Stasiun Kereta Cepat Hali...","Rabu, 08 Mei 2024 19:18 WIB",https://news.detik.com/berita/d-7331666/jelang...,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",kereta cepat whoosh,stasiun halim,long weekend,NaN,NaN,detik
3,KPU Tegaskan Pemilih Tak Terdaftar di DPT Bisa...,13/02/2024,https://nasional.kompas.com/read/2024/02/13/21...,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",KPU,pemilu 2024,Hasyim Asy'ari,NaN,NaN,kompas
4,Kemenag Luncurkan Gerakan Senam Haji Jaga Keta...,2024-04-29,https://nasional.tempo.co/read/1861810/kemenag...,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",Senam Haji,Kemenag,Jemaah Haji,Ibadah Haji,Asrama Haji,tempo


In [14]:
# Drop kolom yang tidak diperlukan
data = data.drop(columns=['Judul', 'Waktu', 'Link', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'source'])
data.head(5)

,Content
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem..."


In [15]:
data.isnull().sum()
data = data.dropna()
data = data.reset_index(drop=True)

In [16]:
import pandas as pd
import re
import string
import numpy as np

def clean_pipeline(text):
    # Validasi input awal
    if pd.isna(text) or str(text).strip().lower() in ['nan', 'none', '']:
        return np.nan

    try:
        text = str(text).lower().strip()

        # Hapus header yang umum dan isi dalam []
        text = re.sub(r'^[a-z\s,.\-]+ - ', '', text)
        text = re.sub(r'simak.*?(di sini|juga video.*)', '', text)
        text = re.sub(r'\[[^\]]*\]', '', text)

        # Hapus branding media dan domain populer
        media_pattern = r'\b(kompas|tempo|cnnindonesia|detik|liputan6|tribunnews|cnbcindonesia|viva|okezone|indozone|beritasatu)(\.com|\.co\.id|\.id|\.co)?\b'
        text = re.sub(media_pattern, '', text)

        # Hapus domain seperti .co, .id, dll (termasuk .co tanpa ekstensi)
        text = re.sub(r'\b\w+\.(co|id|net|org|info|go)\b', '', text)

        # Hapus entitas non-naratif
        non_naratif = ['tv one', 'antv', 'net tv', 'andromeda mercury', 'dwi anggia']
        if any(p in text for p in non_naratif):
            return np.nan

        # Hapus nama hari, bulan, dan angka
        text = re.sub(r'\b(senin|selasa|rabu|kamis|jumat|sabtu|minggu)\b', '', text)
        text = re.sub(r'\b(januari|februari|maret|april|mei|juni|juli|agustus|september|oktober|november|desember)\b', '', text)
        text = re.sub(r'\b\d{1,4}\b', '', text)

        # Hapus kutipan dan semua tanda baca
        text = re.sub(r'[\"“”‘’\'”“]', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Normalisasi spasi
        text = re.sub(r'\s+', ' ', text).strip()

        return text if text else np.nan

    except Exception as e:
        # Tangani semua error dengan fallback aman
        print(f"[ERROR] Cleaning failed: {e} | Text: {text}")
        return np.nan

# Terapkan pipeline dengan aman
data['Cleaning'] = data['Content'].apply(clean_pipeline)

In [17]:
# Lihat hasil
data.head(5)

,Content,Cleaning
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",presiden joko widodo memakai baju adat betawi ...
1,"TEMPO.CO, Jakarta - Amnesty International Indo...",jakarta amnesty international indonesia membeb...
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",stasiun kereta cepat whoosh di halim jakarta t...
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",komisi pemilihan umum kpu ri memastikan pemili...
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",kementerian agama atau kemenag meluncurkan sen...


In [18]:
# Print satu baris acak untuk memeriksa hasil
print(data['Cleaning'].dropna().sample(1).values[0])

direktur tindak pidana siber dirtipidsiber bareskrim polri brigjen himawan bayu aji mengimbau masyarakat tidak mudah tergiur dengan iklan lowongan kerja yang menawarkan pekerjaan mudah dengan penghasilan besar hal itu buntut kasus scam online jaringan internasional dengan modus lowongan kerja paruh waktu himawan menyebut aktivitas kejahatan dengan modus penipuan tersebut terus meningkat setiap tahunnya ini adalah aktivitas online yang cukup tinggi dan dari tahun ke tahun semakin meningkat maka sifatnya transnasional crime yang borderless itu bisa dikendalikan dari mana saja dan itu bisa menimpa kepada siapa saja kata himawan dalam jumpa pers di gedung bareskrim polri jakarta selatan dia mengingatkan masyarakat untuk waspada akan iklan loker yang banyak dibagikan di sosial media penggunaan aktivitas online ini juga harus dibarengi dengan edukasi yang baik sehingga kita selalu berhatihati terhadap segala bentuk penipuan yang pasti nanti akan marak terjadi kata himawan mengingatkan dia ke

In [19]:
import re
import pandas as pd

# Fungsi tokenisasi yang aman dan efisien
def tokenize_indonesian(text):
    try:
        # Validasi input
        if pd.isna(text) or not isinstance(text, str) or text.strip() == '':
            return []

        # Normalisasi ke huruf kecil dan hanya ambil huruf + spasi
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text.lower())

        # Tokenisasi berdasarkan spasi
        tokens = cleaned_text.split()

        return tokens if tokens else []
    
    except Exception as e:
        print(f"[ERROR] Tokenization failed: {e} | Text: {text}")
        return []

# Aplikasikan pada kolom 'Cleaning'
data['Tokenized'] = data['Cleaning'].apply(tokenize_indonesian)

In [20]:
# Implementasikan tokenisasi
data.head(5)

,Content,Cleaning,Tokenized
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",presiden joko widodo memakai baju adat betawi ...,"[presiden, joko, widodo, memakai, baju, adat, ..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo...",jakarta amnesty international indonesia membeb...,"[jakarta, amnesty, international, indonesia, m..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",stasiun kereta cepat whoosh di halim jakarta t...,"[stasiun, kereta, cepat, whoosh, di, halim, ja..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",komisi pemilihan umum kpu ri memastikan pemili...,"[komisi, pemilihan, umum, kpu, ri, memastikan,..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",kementerian agama atau kemenag meluncurkan sen...,"[kementerian, agama, atau, kemenag, meluncurka..."


In [21]:
# Print satu baris acak untuk memeriksa hasil
print(data['Tokenized'].dropna().sample(1).values[0])

['organisasi', 'relawan', 'pendukung', 'jokowi', 'yakni', 'projo', 'buka', 'suara', 'soal', 'dukungan', 'di', 'pilgub', 'jawa', 'barat', 'bendahara', 'umum', 'dpp', 'projo', 'panel', 'barus', 'menyebut', 'organisasinya', 'condong', 'mendukung', 'dedi', 'mulyadipanel', 'mengatakan', 'dukungan', 'projo', 'untuk', 'calon', 'kepala', 'daerah', 'tak', 'terlepas', 'dari', 'keputusan', 'partai', 'politik', 'yang', 'ada', 'di', 'koalisi', 'indonesia', 'maju', 'kim', 'dia', 'mengatakan', 'sudah', 'ada', 'dua', 'partai', 'di', 'kim', 'yakni', 'golkar', 'dan', 'gerindra', 'yang', 'bakal', 'mendukung', 'dedi', 'mulyadi', 'di', 'pilgub', 'jabar', 'karena', 'itu', 'projo', 'memiliki', 'kecenderungan', 'yang', 'sama', 'persen', 'kita', 'ke', 'dedi', 'mulyadi', 'cuma', 'masih', 'perlu', 'diskusi', 'lebih', 'dalam', 'karena', 'projo', 'selalu', 'mengedepankan', 'diskusi', 'dulu', 'kata', 'panel', 'barus', 'di', 'dpp', 'projo', 'jakarta', 'selatan', 'caloncalon', 'yang', 'didukung', 'kim', 'golkar', 'ge

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi stemmer Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Daftar stopwords Anda sudah tersedia di variabel stop_words
stop_words = {
    'yang', 'di', 'ke', 'dengan', 'dan', 'untuk', 'adalah', 'pada', 'ini', 'itu',
    'dari', 'oleh', 'atau', 'karena', 'sebuah', 'tersebut', 'sebagai', 'juga', 'saat',
    'dapat', 'mereka', 'kami', 'kita', 'anda', 'aku', 'saya', 'semua', 'setiap',
    'beberapa', 'banyak', 'sedikit', 'satu', 'dua', 'tiga', 'empat', 'lima', 'enam',
    'tujuh', 'delapan', 'sembilan', 'sepuluh', 'belum', 'sudah', 'akan', 'sedang',
    'telah', 'masih', 'hanya', 'saja', 'lebih', 'kurang', 'sama', 'mungkin',
    'sepertinya', 'seolah', 'sebenarnya', 'terutama', 'termasuk', 'seperti', 'begitu',
    'sangat', 'cukup', 'hampir', 'nyaris', 'sekali', 'kali', 'lain', 'berikutnya',
    'sebelumnya', 'jika', 'bila', 'agar', 'meski', 'walaupun', 'namun', 'tetapi',
    'lalu', 'kemudian', 'hingga', 'bahwa', 'yakni', 'adapun', 'melainkan'
}

# Fungsi filtering yang aman dan efisien + stemming
def filtering(tokens):
    try:
        # Validasi input harus list dan tiap elemen string
        if not isinstance(tokens, list) or not all(isinstance(t, str) for t in tokens):
            return []

        # Filter stopwords dan token kosong
        filtered = [t for t in tokens if t and t.strip() not in stop_words]

        # Lakukan stemming
        stemmed = [stemmer.stem(t) for t in filtered]

        # Deduplikasi sambil mempertahankan urutan
        return list(dict.fromkeys(stemmed))
    
    except Exception as e:
        print(f"[ERROR] Filtering failed: {e}")
        return []

# Terapkan ke kolom DataFrame
data['Filtered'] = data['Tokenized'].apply(lambda tokens: filtering(tokens))


In [23]:
# Terapkan filtering pada DataFrame
data.head(5)

,Content,Cleaning,Tokenized,Filtered
0,"JAKARTA, KOMPAS.com - Presiden Joko Widodo me...",presiden joko widodo memakai baju adat betawi ...,"[presiden, joko, widodo, memakai, baju, adat, ...","[presiden, joko, widodo, memakai, baju, adat, ..."
1,"TEMPO.CO, Jakarta - Amnesty International Indo...",jakarta amnesty international indonesia membeb...,"[jakarta, amnesty, international, indonesia, m...","[jakarta, amnesty, international, indonesia, m..."
2,"Stasiun kereta cepat Whoosh di Halim, Jakarta ...",stasiun kereta cepat whoosh di halim jakarta t...,"[stasiun, kereta, cepat, whoosh, di, halim, ja...","[stasiun, kereta, cepat, whoosh, halim, jakart..."
3,"JAKARTA, KOMPAS.com - Komisi Pemilihan Umum (...",komisi pemilihan umum kpu ri memastikan pemili...,"[komisi, pemilihan, umum, kpu, ri, memastikan,...","[komisi, pemilihan, umum, kpu, ri, memastikan,..."
4,"TEMPO.CO, Jakarta - Kementerian Agama atau Kem...",kementerian agama atau kemenag meluncurkan sen...,"[kementerian, agama, atau, kemenag, meluncurka...","[kementerian, agama, kemenag, meluncurkan, sen..."


In [24]:
# Print satu baris acak untuk memeriksa hasil
print(data['Filtered'].dropna().sample(1).values[0])

['muhammad', 'fajri', 'ilhamsyah', 'salah', 'satu', 'peserta', 'utbk', 'snbt', 'universitas', 'pembangunan', 'nasional', 'veteran', 'jakarta', 'upnvj', 'lantaran', 'tak', 'teliti', 'melihat', 'jadwal', 'tes', 'datang', 'lebih', 'cepat', 'sehari', 'seharusnya', 'saya', 'sudah', 'masuk', 'ruang', 'ujian', 'tapi', 'tidak', 'ada', 'nama', 'ketika', 'cek', 'ulang', 'ternyata', 'baru', 'besok', 'ujiannya', 'ujar', 'kepada', 'panitia', 'terpopuler', 'sosok', 'taipan', 'gang', 'ye', 'diduga', 'berikan', 'fasilitas', 'jet', 'pribadi', 'kaesangerina', 'daftar', 'orang', 'super', 'kaya', 'pakai', 'gulfstream', 'saat', 'dicek', 'terjadwal', 'mengikuti', 'mengaku', 'mantap', 'maka', 'dia', 'beralasan', 'hadir', 'agar', 'terlambat', 'terlalu', 'mempersiapkan', 'diri', 'jadi', 'tadi', 'pagi', 'langsung', 'berangkat', 'berkas', 'siapkan', 'jauhjauh', 'hari', 'masih', 'ucap', 'sambil', 'tersipu', 'malu', 'dilansir', 'laman', 'ahad', 'pengakuan', 'turut', 'diapreasiasi', 'semangatnya', 'luar', 'biasa', 